In [ ]:
from google.colab import drive
import pandas as pd
import re
import numpy as np
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gc
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import pickle
import psycopg2
from sqlalchemy import create_engine
import json
import gzip


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
movies_df = pd.read_csv('gdrive/My Drive/Movies_list.csv',)
movies_df.drop(columns='Unnamed: 0',inplace=True)
movies_df

,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,"Action, Comedy, Crime",An Interpol agent tracks the world's most want...,Rawson Marshall Thurber,"Dwayne Johnson, Ryan Reynolds, Gal Gadot, Ritu..."
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,"Action, Adventure, Fantasy","Shang-Chi, the master of weaponry-based Kung F...",Destin Daniel Cretton,"Simu Liu, Awkwafina, Tony Chiu-Wai Leung, Ben ..."
2,Eternals,2021,6.8,"Action, Adventure, Fantasy","The saga of the Eternals, a race of immortal b...",Chloé Zhao,"Gemma Chan, Richard Madden, Angelina Jolie, Sa..."
3,Dune,2021,8.2,"Action, Adventure, Drama",Feature adaptation of Frank Herbert's science ...,Denis Villeneuve,"Timothée Chalamet, Rebecca Ferguson, Zendaya, ..."
4,No Time to Die,2021,7.4,"Action, Adventure, Thriller",James Bond has left active service. His peace ...,Cary Joji Fukunaga,"Daniel Craig, Ana de Armas, Rami Malek, Léa Se..."
...,...,...,...,...,...,...,...
60991,Grumpy Old New Year,2006 TV Movie,7.2,"Documentary, Comedy",Add a Plot,"Stars:Judith Holder, Rhona Cam...",0
60992,Charlie Cobb's Flash Bash,2008,7.6,Comedy,"At an exclusive Hollywood party, Kirby David (...",Scott Zakarin,"Tim Pilleri, Sabrina Renata Maahs, Maggie Gwin..."
60993,Sofia for Now,2006,8.3,"Comedy, Drama",A young Latina girl named Sofia who goes on a ...,Claudia Duran,"Jessamyn Ansary, William Bailey, Carla Betanco..."
60994,Sex: How It Works,2013 TV Movie,6.5,Documentary,Add a Plot,"Directors:Adam Hopkins, Sue Learoyd, Andre...","Matt Reeves, Alia Ataya, Beckster, Jackson Kir..."


In [ ]:
def only_years(column):
  text = str(column)
  numbers = re.sub('[^0-9]','',text)
  return numbers

In [ ]:
movies_df['Year'] = movies_df['Year'].apply(only_years)
movies_df
del only_years
gc.collect()

117

In [ ]:
movies_df.isnull().sum()

Movie_Title      0
Year             0
Ratings          0
Genre            0
Description      0
Director       353
Cast             0
dtype: int64

In [ ]:
movies_df = movies_df.iloc[0:10000]
movies_df

,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,"Action, Comedy, Crime",An Interpol agent tracks the world's most want...,Rawson Marshall Thurber,"Dwayne Johnson, Ryan Reynolds, Gal Gadot, Ritu..."
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,"Action, Adventure, Fantasy","Shang-Chi, the master of weaponry-based Kung F...",Destin Daniel Cretton,"Simu Liu, Awkwafina, Tony Chiu-Wai Leung, Ben ..."
2,Eternals,2021,6.8,"Action, Adventure, Fantasy","The saga of the Eternals, a race of immortal b...",Chloé Zhao,"Gemma Chan, Richard Madden, Angelina Jolie, Sa..."
3,Dune,2021,8.2,"Action, Adventure, Drama",Feature adaptation of Frank Herbert's science ...,Denis Villeneuve,"Timothée Chalamet, Rebecca Ferguson, Zendaya, ..."
4,No Time to Die,2021,7.4,"Action, Adventure, Thriller",James Bond has left active service. His peace ...,Cary Joji Fukunaga,"Daniel Craig, Ana de Armas, Rami Malek, Léa Se..."
...,...,...,...,...,...,...,...
9995,The River Why,2010,5.6,"Drama, Romance",A young man abandons his family for a solitary...,Matthew Leutwyler,"Zach Gilford, Amber Heard, William Hurt, Dalla..."
9996,Remarkable Shades of Gay,2016,7.3,"Action, Drama, Romance",Remarkable Shades of Gay is a compilation of n...,NaN,0
9997,My Little Sister,2020,6.6,Drama,Lisa has bid goodbye to her ambitions as a pla...,"Directors:Stéphanie Chuat, Véronique Reymond","Nina Hoss, Lars Eidinger, Marthe Keller, Jens ..."
9998,Happily Ever After,2016,5.6,Drama,Heather returns to her hometown when her fathe...,Joan Carr-Wiggin,"Janet Montgomery, Sara Paxton, Alex Kingston, ..."


In [ ]:
movies_df['Director'] = movies_df['Director'].apply(lambda x: re.sub('Directors:','',str(x)))
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,"Action, Comedy, Crime",An Interpol agent tracks the world's most want...,Rawson Marshall Thurber,"Dwayne Johnson, Ryan Reynolds, Gal Gadot, Ritu..."
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,"Action, Adventure, Fantasy","Shang-Chi, the master of weaponry-based Kung F...",Destin Daniel Cretton,"Simu Liu, Awkwafina, Tony Chiu-Wai Leung, Ben ..."
2,Eternals,2021,6.8,"Action, Adventure, Fantasy","The saga of the Eternals, a race of immortal b...",Chloé Zhao,"Gemma Chan, Richard Madden, Angelina Jolie, Sa..."
3,Dune,2021,8.2,"Action, Adventure, Drama",Feature adaptation of Frank Herbert's science ...,Denis Villeneuve,"Timothée Chalamet, Rebecca Ferguson, Zendaya, ..."
4,No Time to Die,2021,7.4,"Action, Adventure, Thriller",James Bond has left active service. His peace ...,Cary Joji Fukunaga,"Daniel Craig, Ana de Armas, Rami Malek, Léa Se..."
...,...,...,...,...,...,...,...
9995,The River Why,2010,5.6,"Drama, Romance",A young man abandons his family for a solitary...,Matthew Leutwyler,"Zach Gilford, Amber Heard, William Hurt, Dalla..."
9996,Remarkable Shades of Gay,2016,7.3,"Action, Drama, Romance",Remarkable Shades of Gay is a compilation of n...,nan,0
9997,My Little Sister,2020,6.6,Drama,Lisa has bid goodbye to her ambitions as a pla...,"Stéphanie Chuat, Véronique Reymond","Nina Hoss, Lars Eidinger, Marthe Keller, Jens ..."
9998,Happily Ever After,2016,5.6,Drama,Heather returns to her hometown when her fathe...,Joan Carr-Wiggin,"Janet Montgomery, Sara Paxton, Alex Kingston, ..."


In [ ]:
movies_df[movies_df['Cast']=='0'].value_counts()

Movie_Title                                             Year  Ratings  Genre                     Description                                                                                                                                                                                                                         Director                                                                              Cast
Yellowstone in Four Seasons                             2017  7.4      Family                    A look at Yellowstone National Park's wildlife, filmed in all four seasons, accompanied by minimal narration and a solo-piano score.                                                                                                    Daniel P. Dauterive                                                               0       1
Welcome to the Blumhouse                                2020  4.8      Horror, Thriller          An interactive program based on the film series developed and prod

In [ ]:
movies_df['Cast'].replace('0',np.NaN,inplace=True)
movies_df.isnull().sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Movie_Title    0
Year           0
Ratings        0
Genre          0
Description    0
Director       0
Cast           9
dtype: int64

In [ ]:
movies_df.dropna(inplace=True)
movies_df.isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Movie_Title    0
Year           0
Ratings        0
Genre          0
Description    0
Director       0
Cast           0
dtype: int64

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie_Title  9991 non-null   object 
 1   Year         9991 non-null   object 
 2   Ratings      9991 non-null   float64
 3   Genre        9991 non-null   object 
 4   Description  9991 non-null   object 
 5   Director     9991 non-null   object 
 6   Cast         9991 non-null   object 
dtypes: float64(1), object(6)
memory usage: 624.4+ KB


In [ ]:
movies_df['Year'].value_counts()

2019    799
2021    783
2020    742
2018    725
2017    631
2016    561
2015    533
2014    503
2013    461
2012    401
2011    382
2009    360
2010    336
2008    336
2007    331
2006    321
2005    290
2004    273
2001    255
2002    254
2003    241
2000    240
1999    232
          1
Name: Year, dtype: int64

In [ ]:
movies_df['Year'] = movies_df['Year'].replace('',np.NaN)
movies_df.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
movies_df['Year'] = pd.to_numeric(movies_df['Year'])
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,"Action, Comedy, Crime",An Interpol agent tracks the world's most want...,Rawson Marshall Thurber,"Dwayne Johnson, Ryan Reynolds, Gal Gadot, Ritu..."
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,"Action, Adventure, Fantasy","Shang-Chi, the master of weaponry-based Kung F...",Destin Daniel Cretton,"Simu Liu, Awkwafina, Tony Chiu-Wai Leung, Ben ..."
2,Eternals,2021,6.8,"Action, Adventure, Fantasy","The saga of the Eternals, a race of immortal b...",Chloé Zhao,"Gemma Chan, Richard Madden, Angelina Jolie, Sa..."
3,Dune,2021,8.2,"Action, Adventure, Drama",Feature adaptation of Frank Herbert's science ...,Denis Villeneuve,"Timothée Chalamet, Rebecca Ferguson, Zendaya, ..."
4,No Time to Die,2021,7.4,"Action, Adventure, Thriller",James Bond has left active service. His peace ...,Cary Joji Fukunaga,"Daniel Craig, Ana de Armas, Rami Malek, Léa Se..."
...,...,...,...,...,...,...,...
9994,It Stains the Sands Red,2016,5.5,"Drama, Horror, Sci-Fi","In the throes of a zombie apocalypse, a troubl...",Colin Minihan,"Brittany Allen, Juan Riedinger, Merwin Mondesi..."
9995,The River Why,2010,5.6,"Drama, Romance",A young man abandons his family for a solitary...,Matthew Leutwyler,"Zach Gilford, Amber Heard, William Hurt, Dalla..."
9997,My Little Sister,2020,6.6,Drama,Lisa has bid goodbye to her ambitions as a pla...,"Stéphanie Chuat, Véronique Reymond","Nina Hoss, Lars Eidinger, Marthe Keller, Jens ..."
9998,Happily Ever After,2016,5.6,Drama,Heather returns to her hometown when her fathe...,Joan Carr-Wiggin,"Janet Montgomery, Sara Paxton, Alex Kingston, ..."


In [ ]:
movies_df[movies_df['Genre']=='0'].value_counts()

Movie_Title            Year  Ratings  Genre  Description                                                                                                                                                                                                                                    Director                        Cast                                                                 
The Christmas Promise  2021  6.9      0      A woman whose fiancé is killed during the Christmas season gets to know the carpenter she hired to finish their new home so she can sell it. Meanwhile, a text she sent to her fiancé's old number pouring out her grief is answered.              Fred Gerber                 Torrey DeVitto, Dylan Bruce, Patrick Duffy, Giles Panton                 1
The Christmas Pitch    2021  7.7      0      When two executives, Becca and William, at a top marketing firm in Chicago are sent to William's hometown of Kringle Lake to rediscover the spirit of Christmas - they begin to fi

In [ ]:
movies_df['Genre'].replace('0',np.NaN,inplace=True)
movies_df.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9986 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie_Title  9986 non-null   object 
 1   Year         9986 non-null   int64  
 2   Ratings      9986 non-null   float64
 3   Genre        9986 non-null   object 
 4   Description  9986 non-null   object 
 5   Director     9986 non-null   object 
 6   Cast         9986 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 624.1+ KB


In [ ]:
movies_df

,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,"Action, Comedy, Crime",An Interpol agent tracks the world's most want...,Rawson Marshall Thurber,"Dwayne Johnson, Ryan Reynolds, Gal Gadot, Ritu..."
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,"Action, Adventure, Fantasy","Shang-Chi, the master of weaponry-based Kung F...",Destin Daniel Cretton,"Simu Liu, Awkwafina, Tony Chiu-Wai Leung, Ben ..."
2,Eternals,2021,6.8,"Action, Adventure, Fantasy","The saga of the Eternals, a race of immortal b...",Chloé Zhao,"Gemma Chan, Richard Madden, Angelina Jolie, Sa..."
3,Dune,2021,8.2,"Action, Adventure, Drama",Feature adaptation of Frank Herbert's science ...,Denis Villeneuve,"Timothée Chalamet, Rebecca Ferguson, Zendaya, ..."
4,No Time to Die,2021,7.4,"Action, Adventure, Thriller",James Bond has left active service. His peace ...,Cary Joji Fukunaga,"Daniel Craig, Ana de Armas, Rami Malek, Léa Se..."
...,...,...,...,...,...,...,...
9994,It Stains the Sands Red,2016,5.5,"Drama, Horror, Sci-Fi","In the throes of a zombie apocalypse, a troubl...",Colin Minihan,"Brittany Allen, Juan Riedinger, Merwin Mondesi..."
9995,The River Why,2010,5.6,"Drama, Romance",A young man abandons his family for a solitary...,Matthew Leutwyler,"Zach Gilford, Amber Heard, William Hurt, Dalla..."
9997,My Little Sister,2020,6.6,Drama,Lisa has bid goodbye to her ambitions as a pla...,"Stéphanie Chuat, Véronique Reymond","Nina Hoss, Lars Eidinger, Marthe Keller, Jens ..."
9998,Happily Ever After,2016,5.6,Drama,Heather returns to her hometown when her fathe...,Joan Carr-Wiggin,"Janet Montgomery, Sara Paxton, Alex Kingston, ..."


In [ ]:
movies_df['Director'] = movies_df['Director'].apply(lambda x: str.lower(x.replace(' ','')))
movies_df['Cast'] = movies_df['Cast'].apply(lambda x: str.lower(x.replace(' ','')))
movies_df.reset_index(inplace=True)
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,0,Red Notice,2021,6.4,"Action, Comedy, Crime",An Interpol agent tracks the world's most want...,rawsonmarshallthurber,"dwaynejohnson,ryanreynolds,galgadot,rituarya"
1,1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,"Action, Adventure, Fantasy","Shang-Chi, the master of weaponry-based Kung F...",destindanielcretton,"simuliu,awkwafina,tonychiu-waileung,benkingsley"
2,2,Eternals,2021,6.8,"Action, Adventure, Fantasy","The saga of the Eternals, a race of immortal b...",chloézhao,"gemmachan,richardmadden,angelinajolie,salmahayek"
3,3,Dune,2021,8.2,"Action, Adventure, Drama",Feature adaptation of Frank Herbert's science ...,denisvilleneuve,"timothéechalamet,rebeccaferguson,zendaya,oscar..."
4,4,No Time to Die,2021,7.4,"Action, Adventure, Thriller",James Bond has left active service. His peace ...,caryjojifukunaga,"danielcraig,anadearmas,ramimalek,léaseydoux"
...,...,...,...,...,...,...,...,...
9981,9994,It Stains the Sands Red,2016,5.5,"Drama, Horror, Sci-Fi","In the throes of a zombie apocalypse, a troubl...",colinminihan,"brittanyallen,juanriedinger,merwinmondesir,kri..."
9982,9995,The River Why,2010,5.6,"Drama, Romance",A young man abandons his family for a solitary...,matthewleutwyler,"zachgilford,amberheard,williamhurt,dallasroberts"
9983,9997,My Little Sister,2020,6.6,Drama,Lisa has bid goodbye to her ambitions as a pla...,"stéphaniechuat,véroniquereymond","ninahoss,larseidinger,marthekeller,jensalbinus"
9984,9998,Happily Ever After,2016,5.6,Drama,Heather returns to her hometown when her fathe...,joancarr-wiggin,"janetmontgomery,sarapaxton,alexkingston,peterf..."


In [ ]:
movies_df['Director'] = movies_df['Director'].apply(lambda x: x.replace(',',' '))
movies_df['Cast'] = movies_df['Cast'].apply(lambda x: x.replace(',',' '))
movies_df['Genre'] = movies_df['Genre'].apply(lambda x: str.lower(x.replace(',',' ')))
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,index,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,0,Red Notice,2021,6.4,action comedy crime,An Interpol agent tracks the world's most want...,rawsonmarshallthurber,dwaynejohnson ryanreynolds galgadot rituarya
1,1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,action adventure fantasy,"Shang-Chi, the master of weaponry-based Kung F...",destindanielcretton,simuliu awkwafina tonychiu-waileung benkingsley
2,2,Eternals,2021,6.8,action adventure fantasy,"The saga of the Eternals, a race of immortal b...",chloézhao,gemmachan richardmadden angelinajolie salmahayek
3,3,Dune,2021,8.2,action adventure drama,Feature adaptation of Frank Herbert's science ...,denisvilleneuve,timothéechalamet rebeccaferguson zendaya oscar...
4,4,No Time to Die,2021,7.4,action adventure thriller,James Bond has left active service. His peace ...,caryjojifukunaga,danielcraig anadearmas ramimalek léaseydoux
...,...,...,...,...,...,...,...,...
9981,9994,It Stains the Sands Red,2016,5.5,drama horror sci-fi,"In the throes of a zombie apocalypse, a troubl...",colinminihan,brittanyallen juanriedinger merwinmondesir kri...
9982,9995,The River Why,2010,5.6,drama romance,A young man abandons his family for a solitary...,matthewleutwyler,zachgilford amberheard williamhurt dallasroberts
9983,9997,My Little Sister,2020,6.6,drama,Lisa has bid goodbye to her ambitions as a pla...,stéphaniechuat véroniquereymond,ninahoss larseidinger marthekeller jensalbinus
9984,9998,Happily Ever After,2016,5.6,drama,Heather returns to her hometown when her fathe...,joancarr-wiggin,janetmontgomery sarapaxton alexkingston peterf...


In [ ]:
lem = WordNetLemmatizer()
def clean_text(text):
  text = text.lower()
  text = text.translate(str.maketrans('','',string.punctuation))
  text = [lem.lemmatize(w) for w in text.split() if w not in stopwords.words('english')]
  text = ' '.join(text)
  return text

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
movies_df['Description'] = movies_df['Description'].apply(clean_text)
movies_df.drop(columns='index',inplace=True)
del clean_text
gc.collect()
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,action comedy crime,interpol agent track world wanted art thief,rawsonmarshallthurber,dwaynejohnson ryanreynolds galgadot rituarya
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,action adventure fantasy,shangchi master weaponrybased kung fu forced c...,destindanielcretton,simuliu awkwafina tonychiu-waileung benkingsley
2,Eternals,2021,6.8,action adventure fantasy,saga eternals race immortal being lived earth ...,chloézhao,gemmachan richardmadden angelinajolie salmahayek
3,Dune,2021,8.2,action adventure drama,feature adaptation frank herbert science ficti...,denisvilleneuve,timothéechalamet rebeccaferguson zendaya oscar...
4,No Time to Die,2021,7.4,action adventure thriller,james bond left active service peace shortlive...,caryjojifukunaga,danielcraig anadearmas ramimalek léaseydoux
...,...,...,...,...,...,...,...
9981,It Stains the Sands Red,2016,5.5,drama horror sci-fi,throe zombie apocalypse troubled woman la vega...,colinminihan,brittanyallen juanriedinger merwinmondesir kri...
9982,The River Why,2010,5.6,drama romance,young man abandon family solitary life flyfish...,matthewleutwyler,zachgilford amberheard williamhurt dallasroberts
9983,My Little Sister,2020,6.6,drama,lisa bid goodbye ambition playwright berlin ar...,stéphaniechuat véroniquereymond,ninahoss larseidinger marthekeller jensalbinus
9984,Happily Ever After,2016,5.6,drama,heather return hometown father becomes ill pre...,joancarr-wiggin,janetmontgomery sarapaxton alexkingston peterf...


In [ ]:
def combine(dataframe):
  bag = dataframe['Genre'] + ' ' + dataframe['Director'] + ' ' + dataframe['Cast'] + ' ' + dataframe['Description']
  return bag

In [ ]:
movies_df

,Movie_Title,Year,Ratings,Genre,Description,Director,Cast
0,Red Notice,2021,6.4,action comedy crime,interpol agent track world wanted art thief,rawsonmarshallthurber,dwaynejohnson ryanreynolds galgadot rituarya
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,action adventure fantasy,shangchi master weaponrybased kung fu forced c...,destindanielcretton,simuliu awkwafina tonychiu-waileung benkingsley
2,Eternals,2021,6.8,action adventure fantasy,saga eternals race immortal being lived earth ...,chloézhao,gemmachan richardmadden angelinajolie salmahayek
3,Dune,2021,8.2,action adventure drama,feature adaptation frank herbert science ficti...,denisvilleneuve,timothéechalamet rebeccaferguson zendaya oscar...
4,No Time to Die,2021,7.4,action adventure thriller,james bond left active service peace shortlive...,caryjojifukunaga,danielcraig anadearmas ramimalek léaseydoux
...,...,...,...,...,...,...,...
9981,It Stains the Sands Red,2016,5.5,drama horror sci-fi,throe zombie apocalypse troubled woman la vega...,colinminihan,brittanyallen juanriedinger merwinmondesir kri...
9982,The River Why,2010,5.6,drama romance,young man abandon family solitary life flyfish...,matthewleutwyler,zachgilford amberheard williamhurt dallasroberts
9983,My Little Sister,2020,6.6,drama,lisa bid goodbye ambition playwright berlin ar...,stéphaniechuat véroniquereymond,ninahoss larseidinger marthekeller jensalbinus
9984,Happily Ever After,2016,5.6,drama,heather return hometown father becomes ill pre...,joancarr-wiggin,janetmontgomery sarapaxton alexkingston peterf...


In [ ]:
movies_df['Bags'] = movies_df.apply(combine, axis=1)
del combine
gc.collect()
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Movie_Title,Year,Ratings,Genre,Description,Director,Cast,Bags
0,Red Notice,2021,6.4,action comedy crime,interpol agent track world wanted art thief,rawsonmarshallthurber,dwaynejohnson ryanreynolds galgadot rituarya,action comedy crime rawsonmarshallthurber dw...
1,Shang-Chi and the Legend of the Ten Rings,2021,7.6,action adventure fantasy,shangchi master weaponrybased kung fu forced c...,destindanielcretton,simuliu awkwafina tonychiu-waileung benkingsley,action adventure fantasy destindanielcretton...
2,Eternals,2021,6.8,action adventure fantasy,saga eternals race immortal being lived earth ...,chloézhao,gemmachan richardmadden angelinajolie salmahayek,action adventure fantasy chloézhao gemmachan...
3,Dune,2021,8.2,action adventure drama,feature adaptation frank herbert science ficti...,denisvilleneuve,timothéechalamet rebeccaferguson zendaya oscar...,action adventure drama denisvilleneuve timot...
4,No Time to Die,2021,7.4,action adventure thriller,james bond left active service peace shortlive...,caryjojifukunaga,danielcraig anadearmas ramimalek léaseydoux,action adventure thriller caryjojifukunaga d...
...,...,...,...,...,...,...,...,...
9981,It Stains the Sands Red,2016,5.5,drama horror sci-fi,throe zombie apocalypse troubled woman la vega...,colinminihan,brittanyallen juanriedinger merwinmondesir kri...,drama horror sci-fi colinminihan brittanyall...
9982,The River Why,2010,5.6,drama romance,young man abandon family solitary life flyfish...,matthewleutwyler,zachgilford amberheard williamhurt dallasroberts,drama romance matthewleutwyler zachgilford am...
9983,My Little Sister,2020,6.6,drama,lisa bid goodbye ambition playwright berlin ar...,stéphaniechuat véroniquereymond,ninahoss larseidinger marthekeller jensalbinus,drama stéphaniechuat véroniquereymond ninahoss...
9984,Happily Ever After,2016,5.6,drama,heather return hometown father becomes ill pre...,joancarr-wiggin,janetmontgomery sarapaxton alexkingston peterf...,drama joancarr-wiggin janetmontgomery sarapaxt...


In [ ]:
movies_df_database = pd.DataFrame(data=movies_df['Movie_Title'])
movies_df_database

,Movie_Title
0,Red Notice
1,Shang-Chi and the Legend of the Ten Rings
2,Eternals
3,Dune
4,No Time to Die
...,...
9981,It Stains the Sands Red
9982,The River Why
9983,My Little Sister
9984,Happily Ever After


In [ ]:
vectorizer = CountVectorizer(stop_words='english',)#max_features=20000)
count_matrix = vectorizer.fit_transform(movies_df['Bags'])
count_matrix.shape

(9986, 41041)

In [ ]:
cosine_sim = cosine_similarity(count_matrix,count_matrix)
cosine_sim.shape


(9986, 9986)

In [ ]:
with gzip.open('gdrive/My Drive/cosine_sim.pklz','wb') as file1:
  pickle.dump(cosine_sim,file1)
# with open('cosine_sim.json','wb') as file2:
#   json.dump(cosine_sim,file2)

In [ ]:
indices = pd.DataFrame(movies_df.index,index=movies_df.Movie_Title)
# indices['Year'] = movies_df.Year
indices

,0
Movie_Title,
Red Notice,0
Shang-Chi and the Legend of the Ten Rings,1
Eternals,2
Dune,3
No Time to Die,4
...,...
It Stains the Sands Red,9981
The River Why,9982
My Little Sister,9983


In [ ]:
def movie_recommender(title,cosine_sim=cosine_sim):
  indx = indices[0].loc[title]
  sim_scores = list(enumerate(cosine_sim[indx].tolist()))
  sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for i in sim_scores]
  return movies_df['Movie_Title'].iloc[movie_indices]

In [ ]:
movie_recommender('No Time to Die')

27                         Spectre
115              Quantum of Solace
40                   Casino Royale
8152                     High Lane
68                         Skyfall
461        The World Is Not Enough
7607                   Brotherhood
285                Die Another Day
78                    Spider-Man 3
781     xXx: Return of Xander Cage
Name: Movie_Title, dtype: object

In [ ]:
movies_df_database

,Movie_Title
0,Red Notice
1,Shang-Chi and the Legend of the Ten Rings
2,Eternals
3,Dune
4,No Time to Die
...,...
9981,It Stains the Sands Red
9982,The River Why
9983,My Little Sister
9984,Happily Ever After


In [ ]:
indx = indices[0].loc['Legend of the Fist: The Return of Chen Zhen']
print(indx)

9985


In [ ]:
movies_df.duplicated().sum()

0

In [ ]:
let = list(enumerate(cosine_sim[9965]))
let

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.07071067811865475),
 (4, 0.0),
 (5, 0.0),
 (6, 0.03340765523905304),
 (7, 0.0),
 (8, 0.039528470752104736),
 (9, 0.06804138174397717),
 (10, 0.2041241452319315),
 (11, 0.059761430466719674),
 (12, 0.1414213562373095),
 (13, 0.12909944487358055),
 (14, 0.0),
 (15, 0.11858541225631421),
 (16, 0.0),
 (17, 0.06804138174397717),
 (18, 0.0),
 (19, 0.07905694150420947),
 (20, 0.06565321642986127),
 (21, 0.0),
 (22, 0.039528470752104736),
 (23, 0.09128709291752767),
 (24, 0.0),
 (25, 0.0),
 (26, 0.11058146711617285),
 (27, 0.0),
 (28, 0.06804138174397717),
 (29, 0.0),
 (30, 0.04055535528269063),
 (31, 0.14433756729740646),
 (32, 0.09682458365518541),
 (33, 0.034020690871988585),
 (34, 0.036860489038724284),
 (35, 0.029880715233359837),
 (36, 0.03466876226407682),
 (37, 0.08718572905786445),
 (38, 0.09128709291752767),
 (39, 0.10606601717798213),
 (40, 0.0),
 (41, 0.1270001270001905),
 (42, 0.041666666666666664),
 (43, 0.039528470752104736),
 (44, 0.0),
 

In [ ]:
sort = sorted(let,key=lambda x: x[1],reverse=True)
# recom = sort[1:11]
print(sort)

[(9965, 0.9999999999999999), (5079, 0.29488391230979427), (9302, 0.28284271247461895), (4712, 0.27695585470349865), (5433, 0.27499999999999997), (8770, 0.2721655269759087), (4832, 0.2638224265055431), (9948, 0.2638224265055431), (4502, 0.2626128657194451), (5710, 0.2581988897471611), (9555, 0.25259074277046134), (3647, 0.25), (8610, 0.24999999999999997), (8370, 0.2474873734152916), (5015, 0.24618298195866548), (4614, 0.23717082451262841), (7022, 0.236227795630767), (401, 0.23570226039551584), (2129, 0.2338535866733713), (2569, 0.2338535866733713), (6722, 0.2338535866733713), (9657, 0.23145502494313785), (1780, 0.22978625750451445), (2608, 0.22978625750451445), (2937, 0.22978625750451445), (8987, 0.22821773229381917), (5575, 0.22821773229381914), (9737, 0.22821773229381914), (7703, 0.22645540682891913), (3355, 0.2261335084333227), (2728, 0.22613350843332267), (3179, 0.22592402852876595), (9199, 0.22592402852876592), (6404, 0.22225022225033336), (4009, 0.22116293423234573), (6699, 0.2211

In [ ]:
no = [n[0] for n in recom]
no

[5079, 9302, 4712, 5433, 8770, 4832, 9948, 4502, 5710, 9555]

In [ ]:
movies_df['Movie_Title'].iloc[no]

5079                       Underdog
9302              The Boys Are Back
4712                      Severance
5433                         Allure
8770           Cooking Up Christmas
4832                 Calendar Girls
9948                          Whore
4502               Return to Sender
5710                 Into the White
9555    Elvis Has Left the Building
Name: Movie_Title, dtype: object